In [52]:
result = reader.readtext("images/convo.png",paragraph="True")
result

[[[[105, 26], [323, 26], [323, 65], [105, 65]], 'Messages File'],
 [[[357, 26], [413, 26], [413, 56], [357, 56]], 'Edit'],
 [[[450, 30], [512, 30], [512, 56], [450, 56]], 'View'],
 [[[550, 30], [715, 30], [715, 59], [550, 59]], 'Conversation'],
 [[[749, 30], [853, 30], [853, 56], [749, 56]], 'Window'],
 [[[884, 26], [946, 26], [946, 59], [884, 59]], 'Help'],
 [[[2454, 25], [2797, 25], [2797, 61], [2454, 61]],
  'Al Goes Local . 2h 49m left'],
 [[[2965, 22], [3240, 22], [3240, 67], [2965, 67]], '@ # @s) 0'],
 [[[3344, 25], [3684, 25], [3684, 62], [3344, 62]],
  'Al Goes Local (2h 49m left)'],
 [[[3843, 26], [4081, 26], [4081, 59], [3843, 59]], 'Sat Sep 5:11PM'],
 [[[681, 122], [963, 122], [963, 160], [681, 160]], 'To: +1 (510) 542-0847'],
 [[[71, 206], [165, 206], [165, 239], [71, 239]], 'Search'],
 [[[2319, 218], [2437, 218], [2437, 250], [2319, 250]], 'iMessage'],
 [[[2247, 241], [2508, 241], [2508, 277], [2247, 277]],
  'Tue, Jul 23 at 2:57PM'],
 [[[142, 283], [602, 283], [602, 381],

In [49]:
import easyocr
reader = easyocr.Reader(['en'])
result = reader.readtext("images/evals.png",paragraph="False")
result

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

/Users/mdbirlikci/Documents/Code Projects/genailocal/venv/lib/python3.11/site-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_

[[[[129, 0], [504, 0], [504, 15], [129, 15]], 'OCEDDCNCAZ Tius Leaueidoalu'],
 [[[601, 0], [952, 0], [952, 17], [601, 17]], 'Mi-denC WAL Leauendoaru'],
 [[[1049, 0], [1281, 0], [1281, 15], [1049, 15]], 'DLIN Leaueidoalu'],
 [[[1378, 0], [1629, 0], [1629, 20], [1378, 20]], 'Dericn Leauendoaru'],
 [[[1727, 0], [1975, 0], [1975, 15], [1727, 15]], 'Adencn LeauenDoaru'],
 [[[2070, 0], [2317, 0], [2317, 17], [2070, 17]], 'MTVQA Leauendoaru'],
 [[[90, 56], [548, 56], [548, 156], [90, 156]],
  'Main Evaluation Results Metrics:'],
 [[[127, 176], [1800, 176], [1800, 455], [127, 455]],
  'Avg = Score: The average score on all VLM Benchmarks (normalized to 0 - 100,the higher the better). Avg Rank: The average rank on all VLM Benchmarks (the lower the better). Avg Score & Rank are calculated based on selected benchmark When results for some selected benchmarks are missing; Avg Score Rank will be Nonelll By default; we present the overall evaluation results based on 8 VLM benchmarks, sorted by the d

In [29]:
from openai import OpenAI
import base64
from dotenv import load_dotenv
from pydantic import BaseModel
import os

# Load environment variables from .env file
load_dotenv()

# Set the API key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

class ChatMessage(BaseModel):
    sender: str
    message: str

class Conversation(BaseModel):
    person_we_are_corresponding_to: str
    messages: list[ChatMessage]

class GeneralistReasoning(BaseModel):
    source: str
    summary: str
    details: str

client = OpenAI()
class GPTModel:
    def __init__(self):
        # self.model = "gpt-4o-mini-2024-07-18"
        self.model = "gpt-4o-2024-08-06"
        self.client = OpenAI()

    # Function to encode the image
    def encode_image(self, image_path):
        with open(image_path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
        
    def format_inputs_to_message(self, image, prompt):
        formatted_image = image if image.startswith("http") else f"data:image/jpeg;base64,{self.encode_image(image)}"
        image_dict = {"url": formatted_image}
        prompt_dict = {"type": "text", "text": prompt}
        image_dict = {"type": "image_url", "image_url": image_dict}
        user_content = [prompt_dict, image_dict]
        messages = [{"role": "user", "content": user_content}]
        return messages
    
    def forward(self, image, prompt):
        messages = self.format_inputs_to_message(image, prompt)
        response = self.client.beta.chat.completions.parse(model=self.model, messages=messages)
        return response.choices[0]

from prompting import generalist_prompt, conversation_prompt
class GeneralistModel(GPTModel):
    def forward(self, image):
        messages = self.format_inputs_to_message(image, generalist_prompt)
        response = self.client.beta.chat.completions.parse(model=self.model, response_format=GeneralistReasoning, messages=messages)
        return response.choices[0].message.content
    
class ConversationModel(GPTModel):
    def __init__(self):
        super().__init__()
        self.model = "gpt-4o-2024-08-06"

    def forward(self, image):
        messages = self.format_inputs_to_message(image, conversation_prompt)
        response = self.client.beta.chat.completions.parse(model=self.model, response_format=Conversation, messages=messages, temperature=0.2)
        return response.choices[0].message.content


    


In [32]:
general_agent = GeneralistModel()
conversation_agent = ConversationModel()

In [31]:
conversation_agent.forward("images/image.png")

'{"person_we_are_corresponding_to":"+1 (510) 542-0847","messages":[{"sender":"Me","message":"Kamal congrats on YC dude! Hope you’re doing good :)"},{"sender":"+1 (510) 542-0847","message":"Thank you Mehmet !"},{"sender":"+1 (510) 542-0847","message":"Sorry for missing this"},{"sender":"+1 (510) 542-0847","message":"Just launched :)"}]}'

In [ ]:
import json

In [33]:
import json

response = general_agent.forward("images/calendar.png")
json_response = json.loads(response)
json_response

{'source': 'Google Calendar',
 'summary': 'Calendar events for September 7th, 2024',
 'details': "Event 1: 'Cari!' | Date: 2024-09-07 | Start: 4:00 AM | End: 6:00 AM\nEvent 2: 'Weekly Reflection' | Date: 2024-09-07 | Start: 4:00 AM | End: 4:45 AM\nEvent 3: 'Yamasun' | Date: 2024-09-07 | Start: 8:00 AM | End: 12:00 PM\nEvent 4: 'AI Goes Local' | Date: 2024-09-07 | Start: 11:00 AM | End: 8:00 PM\nEvent 5: 'Calvin' | Date: 2024-09-07 | Start: 3:30 PM | End: 4:30 PM | Location: 1630 Mission St\nEvent 6: 'SF Startup Series' | Date: 2024-09-07 | Start: 5:00 PM | Location: 414 Oak St\nEvent 7: 'Arya' | Date: 2024-09-07 | Start: 7:00 PM | End: 8:00 PM\nEvent 8: 'Housewarming Dinner' | Date: 2024-09-07 | Start: 7:30 PM | End: 9:30 PM\nEvent 9: 'Elf = Deniz' | Date: 2024-09-09 | 8:00 AM - 8:45 AM"}

In [43]:
import json

response = general_agent.forward("images/evals.png")
json_response = json.loads(response)
json_response

{'source': 'Dashboard - Model Evaluation Results',
 'summary': 'Tabular data of model evaluation results',
 'details': "Header: 'Rank', 'Method', 'Param (B)', 'Language Model', 'Vision Model', 'Avg Score', 'Avg Rank', 'MMBench_V11', 'MMStar', 'MMMU_VAL', 'MathVista', 'OCR'\nRow 1: '1', 'MiniCPM-V-2.6', '8', 'Qwen2-7B', 'SigLIP-400M', '65.2', '13.62', '78', '57.5', '49.8', '60.6', '852'\nRow 2: '2', 'InternVL-2-8B', '8', 'InternLM2.5-7B', 'InternViT-300M', '64.1', '15', '79.4', '61.5', '51.2', '58.3', '794'\nRow 3: '3', 'InternVL-2-4B', '4', 'Phi-3', 'InternViT-300M', '60.6', '25.5', '73.6', '53.9', '48.3', '58.1', '784'\nRow 4: '4', 'InternVL-2-2B', '2', 'InternLM2-1.8B', 'InternViT-300M', '54', '49.5', '69.6', '49.8', '36.3', '46', '781'\nRow 5: '5', 'GLM-4v-9B', '9', 'GLM-4-9B', 'EVA-02-5B', '59.1', '31.75', '67.9', '54.8', '46.9', '51.1', '776'\nRow 6: '26', 'InternVL-1B', '1', 'Qwen2-0.5B', '', '48.3', '', '59.7', '45.6', '36.7', '', '755'\nRow 7: '7', 'Ovisi.5-Llama3-8B', '8', 'Ll

In [45]:
print(_43["details"])

Header: 'Rank', 'Method', 'Param (B)', 'Language Model', 'Vision Model', 'Avg Score', 'Avg Rank', 'MMBench_V11', 'MMStar', 'MMMU_VAL', 'MathVista', 'OCR'
Row 1: '1', 'MiniCPM-V-2.6', '8', 'Qwen2-7B', 'SigLIP-400M', '65.2', '13.62', '78', '57.5', '49.8', '60.6', '852'
Row 2: '2', 'InternVL-2-8B', '8', 'InternLM2.5-7B', 'InternViT-300M', '64.1', '15', '79.4', '61.5', '51.2', '58.3', '794'
Row 3: '3', 'InternVL-2-4B', '4', 'Phi-3', 'InternViT-300M', '60.6', '25.5', '73.6', '53.9', '48.3', '58.1', '784'
Row 4: '4', 'InternVL-2-2B', '2', 'InternLM2-1.8B', 'InternViT-300M', '54', '49.5', '69.6', '49.8', '36.3', '46', '781'
Row 5: '5', 'GLM-4v-9B', '9', 'GLM-4-9B', 'EVA-02-5B', '59.1', '31.75', '67.9', '54.8', '46.9', '51.1', '776'
Row 6: '26', 'InternVL-1B', '1', 'Qwen2-0.5B', '', '48.3', '', '59.7', '45.6', '36.7', '', '755'
Row 7: '7', 'Ovisi.5-Llama3-8B', '8', 'Llama-3-8B-Instruct', 'SigLIP-400M', '62.2', '20', '76.6', '57.7', '48.3', '63', '744'
Row 8: '8', 'MiniCPM-Llama3-V2.5', '8', 'L

In [47]:
import json

response = general_agent.forward("images/calendar.png")
print(json.loads(response))

{'source': 'Google Calendar', 'summary': 'Calendar events for September 7-10, 2024', 'details': "Event 1: Can't Siglo? RushNampa Inka Ei | Date: 2024-09-07 | Start: 8:00 AM | End: 9:00 AM\nEvent 2: Weekly Reflection | Date: 2024-09-07 | Start: 4:45 PM | End: 5:00 PM\nEvent 3: AI Goes Local | Date: 2024-09-07 | Start: 11:00 AM | End: 6:00 PM\nEvent 4: Yarmasun | Date: 2024-09-08 | Start: 8:00 AM | End: 12:00 PM\nEvent 5: Calvin | Date: 2024-09-08 | Start: 2:30 PM | End: 4:30 PM\nEvent 6: SF Startup Series | Date: 2024-09-08 | Start: 5:00 PM | End: 6:00 PM\nEvent 7: Aya | Date: 2024-09-08 | Start: 7:00 PM | End: 8:00 PM\nEvent 8: Elf = Deniz | Date: 2024-09-09 | Start: 8:45 AM | End: 9:00 AM"}


In [48]:
import json

response = general_agent.forward("images/evals.png")
print(json.loads(response))

{'source': 'Web Application - Model Evaluation Results', 'summary': 'Main evaluation results for AI models', 'details': "Header: 'Rank', 'Method', 'Params (B)', 'Language Model', 'Vision Model', 'Avg Score', 'Avg Rank', 'MMBench_V11', 'MMStar', 'MMMU_VAL', 'MathVista', 'OCR'\nRow 1: '1', 'MiniCPM-V2.6', '8', 'Qwen2-7B', 'SigILIP-400M', '65.2', '13.62', '78', '57.5', '49.8', '60.6', '852'\nRow 2: '2', 'InternVL-2-8B', '8', 'InternLM2.5-7B', 'InternViT-300M', '64.1', '15', '79.4', '61.5', '51.2', '58.3', '794'\nRow 3: '3', 'InternVL-2-4B', '4', 'Phi-3', 'InternViT-300M', '60.6', '25.5', '73.6', '53.9', '48.3', '58.1', '784'\nRow 4: '4', 'InternVL-2-2B', '2', 'InternLM2-1.8B', 'InternViT-300M', '54', '49.5', '69.6', '49.8', '36.3', '46', '781'\nRow 5: '5', 'GLM-4v-9B', '9', 'GLM-4.9', 'EVA-02-5B', '59.1', '31.75', '67.9', '54.8', '46.9', '51.1', '776'\nRow 6: '26', 'InternVL-1B', '', 'Qwen2-0.5B', '', '48.3', '69.38', '59.7', '45.6', '36.7', '39.4', '755'\nRow 7: '13', 'Ovisi.5-Llama3-8B'

In [39]:
print(json.loads(response)["details"])

Header: 'Rank', 'Method', 'Param (B)', 'Language Model', 'Vision Model', 'Avg Score', 'Avg Rank', 'MMBench_V11', 'MMStar', 'MMMU_VAL', 'MathVista', 'OCR'
Row 1: '1', 'MiniCPM-V-2.6', '8', 'Qwen2-7B', 'SigLIP-400M', '65.2', '13.62', '78', '57.5', '49.8', '60.6', '852'
Row 2: '5', 'InternLMv2-8B', '8', 'InternLM2.5-7B', 'InternViT-300M', '64.1', '15', '79.4', '61.5', '51.2', '58.3', '794'
Row 3: '5', 'InternLV2-4B', '4', 'Phi-3', 'InternViT-300M', '60.6', '25.5', '73.6', '53.9', '48.3', '58.1', '784'
Row 4: '13', 'InternLV2-2B', '2', 'InternLM2-1.8B', 'InternViT-300M', '54', '49.5', '69.6', '49.8', '36.3', '46', '781'
Row 5: '6', 'GLM-4v-9B', '9', 'GLM-4-9B', 'EVA-02-5B', '59.1', '31.75', '67.9', '54.8', '46.9', '51.1', '776'
Row 6: '26', 'InternLVL2-1B', '1', 'Qwen2-0.5B', 'InternViT-300M', '48.3', '50.9', '59.7', '45.6', '36.7', '39.4', '755'
Row 7: '3', 'Ovisi_5-llama3-8B', '8', 'Llama-3-8B-Instruct', 'SigLIP-400M', '62.2', '20', '76.6', '57.3', '48.3', '63', '744'
Row 9: '7', 'MiniCP

In [22]:
print(gpt.forward("images/calendar.png", "Look at my calendar and output all the events that I have, when they start, and when they end.").message.content)

Here are the events from your calendar for September 7th to September 10th, 2024:

### Saturday, September 7th, 2024
- **Event:** Can’t Stop! (Rush/Mmmpie Knar En)
  - **Start Time:** 1 AM
  - **End Time:** 2 AM
  
- **Event:** Weekly Reflection
  - **Start Time:** 4 AM
  - **End Time:** 4:45 AM

### Sunday, September 8th, 2024
There are no events on this day.

### Monday, September 9th, 2024
- **Event:** Yamusan
  - **Start Time:** 8 AM
  - **End Time:** 12 PM
  
- **Event:** AI Goes Local
  - **Start Time:** 11 AM
  - **End Time:** 8 PM

- **Event:** Calvin
  - **Start Time:** 3:30 PM
  - **End Time:** 4:30 PM
  - **Location:** 1600 Mission St

- **Event:** SF Startup Series
  - **Start Time:** 5 PM
  - **End Time:** 6 PM
  - **Location:** 441 Oak St

- **Event:** Arya
  - **Start Time:** 7 PM
  - **End Time:** 8 PM

- **Event:** Housewarming Dinner
  - **Start Time:** 7:30 PM
  - **End Time:** 9:30 PM

### Tuesday, September 10th, 2024
- **Event:** Eif w/ Deniz
  - **Start Time:** 8

In [20]:
print(gpt.forward("images/evals.png", "Parse all the information on the string, output in terms of text or a markdown format. Do not truncate information.").message.content)

## Main Evaluation Results

### Metrics:
- **Avg Score:** The average score on all VLM Benchmarks (normalized to 0 - 100, the higher the better).
- **Avg Rank:** The average rank on all VLM Benchmarks (the lower the better).
- **Avg Score & Rank** are calculated based on the selected benchmark. When results for some selected benchmarks are missing, Avg Score / Rank will be None.
- By default, the overall evaluation results are based on 8 VLM benchmarks, sorted by the descending order of Avg Score.
  - The following datasets are included in the main results: MMBench_V11, MMStar, MMMU_VAL, MathVista, OCRBench, AID2, HallusionBench, MMVet.
  - Detailed evaluation results for each dataset (included or not included in main) are provided in the consequent tabs.

### Evaluation Dimension

| Avg Score | Avg Rank | MMBench_V11 | MMStar | MME | MMMU_VAL | MathVista | OCRBench | AID2 | HallusionBench | SEEDBench_IMG | LLaVA-Bench | CCBench | RealWorldQA | POPE | ScienceQA_TEST | SEEDBench2_Plus |

In [21]:
prompt = """
You're a helpful assistant to take a screenshot of a UI and explain everything going on in the interface. 

For this image, explain each graph you see. Write a few details on how runs have formulated. 

Imagine that you're explaining this to someone that will never see the image.

Explain each graph and how each run's performance on each graph looks like. 
"""

print(gpt.forward("images/wandb.png", "").message.content)

The screenshot shows a dashboard for visualizing machine learning experiments in a project titled "**hw2p2**" within the workspace of a user called "Denizbirlikci" on a platform that appears to be Weights & Biases (W&B).

Key elements of the screenshot:

1. **Warning Banner**: The user has exceeded 100% of their tracking hours and is prompted to upgrade their plan to continue using W&B without interruption.
   
2. **Project Header**: The project is named "raj_deniz_josh_simrit," and the specific sub-project or notebook in use is "hw2p2."

3. **Workspace Overview**: 

    - The workspace is showing multiple runs (62 total in this example) with a table of the different runs on the left side, each with unique names (e.g., "raj-rewritten-convnxt," "raj-adam-with-warmup," "deniz:viShNet:AdamW-RETRY").
    - Various icons indicate the status, tags, and other attributes of these runs.

4. **Charts/Visualizations**: Six main charts illustrate the results of different runs:
   
    - **Train Ac

In [ ]:
print(gpt.forward("images/calendar.png", "Parse all the information on the string, output in terms of text or a markdown format. Do not truncate information.").message.content)

In [14]:
gpt.forward("images/evals.png", "Parse all the information on the string, output in terms of text or a markdown format. Do not truncate information.")

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='# Main Evaluation Results\n\n## Metrics:\n- **Avg Score**: The average score on all VLM Benchmarks (normalized to 0 - 100, the higher the better).\n- **Avg Rank**: The average rank on all VLM Benchmarks (the lower the better).\n- **Avg Score & Rank** are calculated based on selected benchmark. When results for some selected benchmarks are missing, Avg Score / Rank will be None!!\n\nBy default, we present the overall evaluation results based on 8 VLM benchmarks, sorted by the descending order of Avg Score.\n- The following datasets are included in the main results: MMBench_V11, MMStar, MME, MMU_VAL, MathVista, OCRBench, AID2, HallusionBench, SEEDBench, MMVet.\n- Detailed evaluation results for each dataset (included or not included in main) are provided in the consequent tabs.\n\n## Evaluation Dimension\n- Avg Score\n- Avg Rank\n- MMBench_V11\n- MMStar\n- MME\n- MMU_VAL\n- MathVista\n- OCRBench\n

In [15]:
print(_12.message.content)

Here’s a parsed representation of the information presented in the image:

## Main Evaluation Results

### Metrics
- **Avg Score**: The average score on all VLM Benchmarks (normalized to 0 - 100, the higher the better).
- **Avg Rank**: The average rank on all VLM Benchmarks (the lower the better).
- **Avg Score & Rank**: Calculated based on selected benchmarks. If results for some benchmarks are missing, Avg Score/Rank will be `None`.

### Evaluation Dimension
- The results are organized by descending order of Avg Score.
- The following datasets are included:
  - MMBench_V11
  - MMStar
  - MME
  - MMU_VAL
  - MathVista
  - OCRBench
  - AID2
  - HallusionBench
  - SEEDBench_Plus
  - MMT-Bench_VAL
  - BLINK

### Model Size
- Options are available for <4B, 4B-10B, 10B-20B, 20B-40B, >40B, Unknown.

### Table: Model Performance
| Rank | Method               | Params (B) | Language Model     | Vision Model       | Avg Score | Avg Rank | MMBench_V11 | MMStar | MMU_VAL | MathVista | OCR |
|---